# Return Order Agent

## Imports

In [5]:
# Core libraries
import os
import json
import sqlite3
import pandas as pd

# OpenAI client
from openai import OpenAI


## Tools

In [ ]:
class PDFRetrieverTool:
    def __init__(self, content):
        self.content = content

    def run(self, query: str):
        if "return" in query.lower() or "refund" in query.lower():
            return self.content
        return "No relevant policy found in the PDF."


class DatabaseQueryTool:
    def __init__(self, connection):
        self.conn = connection

    def run(self, query: str):
        try:
            result = pd.read_sql_query(query, self.conn)
            return result.to_dict(orient="records")
        except Exception as e:
            return f"Error executing query: {e}"


## Agent

In [7]:
class AutonomousReturnAgent:
    def __init__(self, llm_client, tools: dict):
        self.client = llm_client
        self.tools = tools
        self.memory = []

    def build_prompt(self, user_message):
        memory_text = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in self.memory])
        return f"""
You are an autonomous customer service AI for Polar, a retail brand.
You can use the following tools when needed:

Tools available:
- pdf_tool: retrieves text from policy PDF.
- db_tool: queries customer and order database using SQL.

Your task is to assist with questions about orders, returns, and refunds.
You decide which tool to call and what SQL or query to run.
Always explain your reasoning and results clearly.

Conversation so far:
{memory_text}

User: {user_message}
If you need to use a tool, respond in JSON with the format:
{{"action": "tool_name", "input": "your query"}}
Otherwise, reply with the final message to the user.
"""

    def process(self, user_message):
        # Append user message to memory
        self.memory.append({"role": "user", "content": user_message})

        # Build reasoning prompt
        prompt = self.build_prompt(user_message)

        # Step 1: Ask LLM what to do
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": "You are a helpful AI agent."},
                      {"role": "user", "content": prompt}]
        )
        message = response.choices[0].message.content.strip()

        # Step 2: If the message looks like a tool call
        try:
            parsed = json.loads(message)
            if "action" in parsed:
                tool_name = parsed["action"]
                tool_input = parsed["input"]
                print(f"🧰 Using tool: {tool_name} → {tool_input}")

                if tool_name not in self.tools:
                    result = f"Unknown tool: {tool_name}"
                else:
                    result = self.tools[tool_name].run(tool_input)

                self.memory.append({"role": "tool", "content": str(result)})

                # Ask LLM to summarize tool result for the user
                follow_up = self.client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {"role": "system", "content": "You are a customer service AI assistant."},
                        {"role": "user", "content": f"The tool returned this: {result}. Summarize or continue the answer to the user clearly."}
                    ]
                )
                final_message = follow_up.choices[0].message.content.strip()
                self.memory.append({"role": "assistant", "content": final_message})
                return final_message
        except Exception:
            pass  # not JSON, continue as normal text

        # Step 3: Otherwise, it’s a normal reply
        self.memory.append({"role": "assistant", "content": message})
        return message


## Chat

In [9]:
os.getcwd()

'c:\\Users\\anaol\\repos\\langchain\\langchain-AI-summit\\notebooks'

In [10]:
import os

pdf_path = "../docs/polar-return_policy.pdf"
print(os.path.exists(pdf_path))  # Should print: True


False


In [8]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../docs/polar-return_policy.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(docs.page_content)

ValueError: File path ../docs/polar-return_policy.pdf is not a valid file or url

In [ ]:
pdf_tool = PDFRetrieverTool(pdf_text)
db_tool = DatabaseQueryTool(conn)

agent = AutonomousReturnAgent(
    llm_client=client,
    tools={"pdf_tool": pdf_tool, "db_tool": db_tool}
)


In [ ]:
print(agent.process("Hi! I want to know how the return policy works."))

print(agent.process("Can you check if order 1001 is eligible for a return?"))

print(agent.process("Who is the customer for order 1002?"))
